<a href="https://colab.research.google.com/github/Swarup012/Skin_disease_image_prediction/blob/master/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"swarupbasu","key":"bf267ec7c48e9b3ab6cb2ec624ae4f27"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

401 Client Error: Unauthorized for url: https://www.kaggle.com/api/v1/datasets/list?group=public&sortby=hottest&size=all&filetype=all&license=all&viewed=unspecified&page=1


In [ ]:
!kaggle datasets download -d pacificrm/skindiseasedataset -p /content/drive/MyDrive/ML

Dataset URL: https://www.kaggle.com/datasets/pacificrm/skindiseasedataset
License(s): CC0-1.0


In [ ]:
!unzip /content/drive/MyDrive/ML/skindiseasedataset.zip -d /content/drive/MyDrive/ML/

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/phototoxic-reactions-lime-juice-2.jpeg  
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/phototoxic-reactions-lime-juice-3.jpeg  
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/sun-damaged-skin-1.jpeg  
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/sun-damaged-skin-10.jpeg  
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/sun-damaged-skin-100.jpeg  
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/sun-damaged-skin-101.jpeg  
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/sun-damaged-skin-102.jpeg  
  inflating: /content/drive/MyDrive/ML/SkinDisease/SkinDisease/train/Sun_Sunlight_Damage/sun-damaged-skin-1

In [2]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

**Here we create the dataframe of the dataset (in this code we join the path with there dir name)**

In [3]:
def create_dataframe(data_dir):
    data = []
    classes = os.listdir(data_dir)
    for class_name in classes:
        class_path = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            data.append({'image_path': img_path, 'label': class_name})
    return pd.DataFrame(data)

**Here we clean the data like delete duplicates, null values and also verify that images are valid**

In [4]:
def clean_dataframe(df):

    df = df.drop_duplicates(subset='image_path')


    df = df.dropna()


    valid_images = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        try:

            with Image.open(row['image_path']) as img:
                img.verify()
            valid_images.append(True)
        except:
            valid_images.append(False)

    df['is_valid'] = valid_images


    df = df[df['is_valid']].drop(columns=['is_valid'])


    df = df.reset_index(drop=True)

    return df

**Cleaning procces started by calling the function**

In [5]:

train_dir = '/content/drive/MyDrive/ML/SkinDisease/SkinDisease/train'
test_dir = '/content/drive/MyDrive/ML/SkinDisease/SkinDisease/test'


train_df = create_dataframe(train_dir)
test_df = create_dataframe(test_dir)


print("Cleaning training data...")
train_df_clean = clean_dataframe(train_df)

print("Cleaning test data...")
test_df_clean = clean_dataframe(test_df)


print(f"\nClean train shape: {train_df_clean.shape}")
print(train_df_clean.head())

print(f"\nClean test shape: {test_df_clean.shape}")
print(test_df_clean.head())

Cleaning training data...


100%|██████████| 13898/13898 [1:28:13<00:00,  2.63it/s]


Cleaning test data...


100%|██████████| 1546/1546 [09:39<00:00,  2.67it/s]


Clean train shape: (13898, 2)
                                          image_path label
0  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
1  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
2  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
3  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
4  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne

Clean test shape: (1546, 2)
                                          image_path label
0  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
1  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
2  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
3  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
4  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne


In [7]:
print(train_df_clean.shape)
print(test_df_clean.shape)

(13898, 2)
(1546, 2)


**Create clean dir in drive**

In [8]:
import shutil

drive_train_dir = '/content/drive/MyDrive/ML/SkinDisease/SkinDisease/cleaned_train'
drive_test_dir = '/content/drive/MyDrive/ML/SkinDisease/SkinDisease/cleaned_test'

os.makedirs(drive_train_dir, exist_ok=True)
os.makedirs(drive_test_dir, exist_ok=True)

**Copy the clean images to the new clean dir**

In [9]:
def copy_clean_images(df, target_dir):
    for label in df['label'].unique():
        label_dir = os.path.join(target_dir, label)
        os.makedirs(label_dir, exist_ok=True)

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        src = row['image_path']
        dst = os.path.join(target_dir, row['label'], os.path.basename(src))
        try:
            shutil.copy(src, dst)
        except Exception as e:
            print(f'Error copying {src}: {e}')


copy_clean_images(train_df_clean, drive_train_dir)
copy_clean_images(test_df_clean, drive_test_dir)


100%|██████████| 1546/1546 [00:44<00:00, 34.56it/s]


**Create the csv file**

In [11]:
train_df_clean.to_csv('/content/drive/MyDrive/ML/SkinDisease/SkinDisease/cleaned_train.csv', index=False)
test_df_clean.to_csv('/content/drive/MyDrive/ML/SkinDisease/SkinDisease/cleaned_test.csv', index=False)

**Run this shell when you open google colab**

In [12]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

train_df_cleaned = pd.read_csv('/content/drive/MyDrive/ML/SkinDisease/SkinDisease/cleaned_train.csv')
test_df_cleaned = pd.read_csv('/content/drive/MyDrive/ML/SkinDisease/SkinDisease/cleaned_test.csv')

print(train_df_cleaned.head())
print(test_df_cleaned.head())


                                          image_path label
0  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
1  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
2  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
3  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
4  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
                                          image_path label
0  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
1  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
2  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
3  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne
4  /content/drive/MyDrive/ML/SkinDisease/SkinDise...  Acne


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import rando